# McNemar Test

## Import All Necessary Libraries

In [11]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar 
import os

## Read All the Data

In [12]:
dir = '/Users/angelaoryza/Documents/TA/noisy-rnnids/rnnids-py/results/vector/mcnemar/smtp'
subdir = os.listdir(dir)

In [13]:
df_dict = {}
for file in subdir:
    if file.endswith('.csv'):
        df = pd.read_csv(f'./{file}')
        file = file.split('.csv')[0]
        df_dict[file] = df

In [14]:
df_dict['bi-lstm_pred_0.0']

,Unnamed: 0,id,b_mean,b_iqr,b_zscore,f_zscore,f_mean,f_iqr
0,0,175.45.176.0-40755-149.171.126.18-25-tcp,0,1,1,1,0,1
1,1,175.45.176.0-60595-149.171.126.17-25-tcp,0,1,1,1,0,1
2,2,175.45.176.2-58472-149.171.126.17-25-tcp,0,1,1,1,0,1
3,3,175.45.176.2-13352-149.171.126.13-25-tcp,0,1,1,1,0,1
4,4,175.45.176.0-3844-149.171.126.12-25-tcp,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...
44743,44743,59.166.0.9-21800-149.171.126.0-25-tcp,0,0,0,0,0,0
44744,44744,59.166.0.4-6591-149.171.126.6-25-tcp,0,0,0,0,0,0
44745,44745,59.166.0.5-7481-149.171.126.2-25-tcp,0,0,0,0,0,0
44746,44746,59.166.0.0-12790-149.171.126.2-25-tcp,0,0,0,0,0,0


In [15]:
for key, df in df_dict.items():
    df.drop('Unnamed: 0', axis=1, inplace=True)

## Data Exploration

In [16]:
for key, df in df_dict.items():
    temp = df[df.duplicated()]['id'].unique()
    print(f'There are {len(temp)} duplicates data in df {key}')

There are 202 duplicates data in df lstm_pred_0.002
There are 203 duplicates data in df lstm_pred_0.003
There are 204 duplicates data in df lstm_pred_0.001
There are 214 duplicates data in df lstm_pred_0.0
There are 197 duplicates data in df lstm_pred_0.004
There are 181 duplicates data in df lstm_pred_0.005
There are 212 duplicates data in df lstm_pred_0.007
There are 194 duplicates data in df lstm_pred_0.006
There are 214 duplicates data in df bi-lstm_pred_0.0
There are 203 duplicates data in df bi-lstm_pred_0.005
There are 201 duplicates data in df bi-lstm_pred_0.004
There are 182 duplicates data in df bi-lstm_pred_0.006
There are 202 duplicates data in df bi-lstm_pred_0.007
There are 193 duplicates data in df bi-lstm_pred_0.003
There are 201 duplicates data in df bi-lstm_pred_0.002
There are 209 duplicates data in df bi-lstm_pred_0.001


In [17]:
for key, df in df_dict.items():
    count = 0
    for t in temp:
        srs = df[df['id']==t][['b_mean', 'b_iqr', 'b_zscore', 'f_mean', 'f_iqr', 'f_zscore' ]].nunique()
        if (srs > 1).any():
            print(t)
            count += 1
            break
        else:
            continue
    if count == 0:
        print(f'All duplicated values in df {key} have the same prediction')
    else:
        print(f'Not all duplicated values in df {key} have the same prediction')

149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.002 have the same prediction
175.45.176.3-18874-149.171.126.17-25-tcp
Not all duplicated values in df lstm_pred_0.003 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.001 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.0 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.004 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.005 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.007 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df lstm_pred_0.006 have the same prediction
149.171.126.8-25-59.166.0.3-52898-tcp
Not all duplicated values in df bi-lstm_pred_0.0 have the same prediction

## Data Pre-Processing

In [18]:
def preprocess(df):
    df = df.sort_values(by='id')
    df.drop_duplicates(inplace=True)
    return df

In [19]:
for key, df in df_dict.items():
    df = preprocess(df)

### Merge DF

In [20]:
df_merge = dict()
x = df_dict['bi-lstm_pred_0.0']
for key, df in df_dict.items():
    if key == 'bi-lstm_pred_0.0':
        continue
    else:
        df_all = pd.merge(x, df, on='id', how='inner')
        df_all = df_all.sort_values(by='id')
        df_merge[key] = df_all

- Plot F1-Score untuk tiap tiap threshold
- Cari trend regresinya, Hitung gradiennya untuk tiap threshold.
- Dilakukan untuk setiap protokol
- Rekap McNemar

- Vectorizing:


## Run McNemar Test

In [21]:
def mcnemar_test(df, x , y, key):
    a = len(df[(df[x]==0) & (df[y]==0)])
    b = len(df[(df[x]==0) & (df[y]==1)])
    c = len(df[(df[x]==1) & (df[y]==0)])
    d = len(df[(df[x]==1) & (df[y]==1)])

    table = np.array([
    [a,b],
    [c,d]
    ])

    print(key)
    print(table)
    result = mcnemar(table)
    print(result)
    return result.pvalue


### For Binary Score

#### For B-Mean

In [22]:
result = {}
result['b-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_mean_x', 'b_mean_y', key)
    result['b-mean'][key] = test

lstm_pred_0.002
[[44549   948]
 [   67    54]]
pvalue      4.911546637646998e-200
statistic   67.0
lstm_pred_0.003
[[44545   952]
 [   72    49]]
pvalue      8.41006313652058e-197
statistic   72.0
lstm_pred_0.001
[[44793   704]
 [   64    57]]
pvalue      3.452567955128791e-137
statistic   64.0
lstm_pred_0.0
[[45445    52]
 [   46    75]]
pvalue      0.6137344676301393
statistic   46.0
lstm_pred_0.004
[[40312  5185]
 [   91    30]]
pvalue      0.0
statistic   91.0
lstm_pred_0.005
[[40322  5175]
 [   91    30]]
pvalue      0.0
statistic   91.0
lstm_pred_0.007
[[40320  5177]
 [   91    30]]
pvalue      0.0
statistic   91.0
lstm_pred_0.006
[[44519   978]
 [   70    51]]
pvalue      1.5031013987231972e-205
statistic   70.0
bi-lstm_pred_0.005
[[44467  1030]
 [   77    44]]
pvalue      1.434274287396423e-213
statistic   77.0
bi-lstm_pred_0.004
[[44582   915]
 [   85    36]]
pvalue      1.8510654440847412e-176
statistic   85.0
bi-lstm_pred_0.006
[[44479  1018]
 [   91    30]]
pvalue      6.42

#### For B-IQR

In [23]:
result['b-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_iqr_x', 'b_iqr_y', key)
    result['b-iqr'][key] = test

lstm_pred_0.002
[[39736    84]
 [  619  5179]]
pvalue      1.3170838673377448e-101
statistic   84.0
lstm_pred_0.003
[[39561   259]
 [  632  5166]]
pvalue      1.0534366555233174e-36
statistic   259.0
lstm_pred_0.001
[[39738    82]
 [  532  5266]]
pvalue      1.0796916455565886e-81
statistic   82.0
lstm_pred_0.0
[[39752    68]
 [  513  5285]]
pvalue      1.822750230985406e-85
statistic   68.0
lstm_pred_0.004
[[39734    86]
 [  253  5545]]
pvalue      3.219245160649999e-20
statistic   86.0
lstm_pred_0.005
[[39065   755]
 [  243  5555]]
pvalue      1.2289092543300596e-61
statistic   243.0
lstm_pred_0.007
[[39353   467]
 [  466  5332]]
pvalue      0.9999999999999998
statistic   466.0
lstm_pred_0.006
[[39365   455]
 [  565  5233]]
pvalue      0.0006357073676317757
statistic   455.0
bi-lstm_pred_0.005
[[39754    66]
 [  602  5196]]
pvalue      3.3119947023694254e-109
statistic   66.0
bi-lstm_pred_0.004
[[39489   331]
 [  587  5211]]
pvalue      2.4298195521269808e-17
statistic   331.0
bi-lst

#### For B-ZScore

In [24]:
result['b-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_zscore_x', 'b_zscore_y', key)
    result['b-zscore'][key] = test

lstm_pred_0.002
[[40148   103]
 [  225  5142]]
pvalue      1.3886133797052441e-11
statistic   103.0
lstm_pred_0.003
[[40141   110]
 [  261  5106]]
pvalue      2.8190909885477205e-15
statistic   110.0
lstm_pred_0.001
[[40152    99]
 [  195  5172]]
pvalue      2.2836945725002502e-08
statistic   99.0
lstm_pred_0.0
[[40177    74]
 [   96  5271]]
pvalue      0.10698657283154894
statistic   74.0
lstm_pred_0.004
[[40139   112]
 [   74  5293]]
pvalue      0.006512203250161622
statistic   74.0
lstm_pred_0.005
[[39874   377]
 [   63  5304]]
pvalue      1.3833650455101763e-55
statistic   63.0
lstm_pred_0.007
[[40139   112]
 [   72  5295]]
pvalue      0.003917460831386664
statistic   72.0
lstm_pred_0.006
[[40140   111]
 [  238  5129]]
pvalue      9.111256955000938e-12
statistic   111.0
bi-lstm_pred_0.005
[[40158    93]
 [  226  5141]]
pvalue      6.32758038850534e-14
statistic   93.0
bi-lstm_pred_0.004
[[40142   109]
 [  222  5145]]
pvalue      5.107233788740849e-10
statistic   109.0
bi-lstm_pred_

### For Floating Score

#### For F-Mean

In [25]:
result['f-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_mean_x', 'f_mean_y', key)
    result['f-mean'][key] = test

lstm_pred_0.002
[[44702   746]
 [  138    32]]
pvalue      1.392860012072931e-101
statistic   138.0
lstm_pred_0.003
[[45038   410]
 [  110    60]]
pvalue      1.1446896361374107e-41
statistic   110.0
lstm_pred_0.001
[[44249  1199]
 [  115    55]]
pvalue      5.386547330431421e-228
statistic   115.0
lstm_pred_0.0
[[45276   172]
 [   41   129]]
pvalue      2.8037311964574213e-20
statistic   41.0
lstm_pred_0.004
[[44914   534]
 [  135    35]]
pvalue      5.372665990986784e-57
statistic   135.0
lstm_pred_0.005
[[43629  1819]
 [  126    44]]
pvalue      0.0
statistic   126.0
lstm_pred_0.007
[[40456  4992]
 [   75    95]]
pvalue      0.0
statistic   75.0
lstm_pred_0.006
[[44814   634]
 [  147    23]]
pvalue      8.187010064513006e-73
statistic   147.0
bi-lstm_pred_0.005
[[44988   460]
 [  116    54]]
pvalue      2.0319755107011e-49
statistic   116.0
bi-lstm_pred_0.004
[[44917   531]
 [  128    42]]
pvalue      3.4095837908091365e-59
statistic   128.0
bi-lstm_pred_0.006
[[44464   984]
 [  148

#### For F-IQR

In [26]:
result['f-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_iqr_x', 'f_iqr_y', key)
    result['f-iqr'][key] = test

lstm_pred_0.002
[[40250   214]
 [  330  4824]]
pvalue      7.479462853412501e-07
statistic   214.0
lstm_pred_0.003
[[40252   212]
 [  451  4703]]
pvalue      9.091877360715193e-21
statistic   212.0
lstm_pred_0.001
[[40033   431]
 [  161  4993]]
pvalue      2.0628200149946204e-29
statistic   161.0
lstm_pred_0.0
[[40282   182]
 [  344  4810]]
pvalue      1.4887520732860009e-12
statistic   182.0
lstm_pred_0.004
[[40072   392]
 [  183  4971]]
pvalue      1.7513605672856608e-18
statistic   183.0
lstm_pred_0.005
[[39580   884]
 [ 2190  2964]]
pvalue      2.1154656570357523e-126
statistic   884.0
lstm_pred_0.007
[[39940   524]
 [  131  5023]]
pvalue      1.5379781361904328e-56
statistic   131.0
lstm_pred_0.006
[[40060   404]
 [  393  4761]]
pvalue      0.7231997072024248
statistic   393.0
bi-lstm_pred_0.005
[[40288   176]
 [  140  5014]]
pvalue      0.04879226376204752
statistic   140.0
bi-lstm_pred_0.004
[[40276   188]
 [  146  5008]]
pvalue      0.024723968235875443
statistic   146.0
bi-lst

#### For F-ZScore

In [27]:
result['f-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_zscore_x', 'f_zscore_y', key)
    result['f-zscore'][key] = test

lstm_pred_0.002
[[40330   264]
 [  368  4656]]
pvalue      4.0303794844461656e-05
statistic   264.0
lstm_pred_0.003
[[40348   246]
 [  507  4517]]
pvalue      1.0458587981969544e-21
statistic   246.0
lstm_pred_0.001
[[40345   249]
 [  196  4828]]
pvalue      0.013610180066168117
statistic   196.0
lstm_pred_0.0
[[40409   185]
 [  283  4741]]
pvalue      6.817059541081604e-06
statistic   185.0
lstm_pred_0.004
[[40413   181]
 [  148  4876]]
pvalue      0.0775363944652636
statistic   148.0
lstm_pred_0.005
[[40185   409]
 [ 2381  2643]]
pvalue      0.0
statistic   409.0
lstm_pred_0.007
[[40201   393]
 [  140  4884]]
pvalue      8.444851882354513e-29
statistic   140.0
lstm_pred_0.006
[[40330   264]
 [  491  4533]]
pvalue      1.1561856589058186e-16
statistic   264.0
bi-lstm_pred_0.005
[[40362   232]
 [  138  4886]]
pvalue      1.1751628541853233e-06
statistic   138.0
bi-lstm_pred_0.004
[[40378   216]
 [  168  4856]]
pvalue      0.016352801511809633
statistic   168.0
bi-lstm_pred_0.006
[[4033

## Saving Test Result

In [28]:
mcnemar_result = pd.DataFrame.from_dict(result).sort_index(axis=0)
mcnemar_result

,b-mean,b-iqr,b-zscore,f-mean,f-iqr,f-zscore
bi-lstm_pred_0.001,2.992670e-156,7.015747e-40,1.166037e-08,8.939432e-213,5.914964e-01,6.171207e-01
bi-lstm_pred_0.002,1.819018e-173,4.102402e-87,1.983461e-06,6.181374e-133,1.241843e-14,1.215324e-99
bi-lstm_pred_0.003,1.511723e-203,4.454664e-02,6.403591e-02,6.739243e-182,2.421343e-189,7.300215e-01
bi-lstm_pred_0.004,1.851065e-176,2.429820e-17,5.107234e-10,3.409584e-59,2.472397e-02,1.635280e-02
bi-lstm_pred_0.005,1.434274e-213,3.311995e-109,6.327580e-14,2.031976e-49,4.879226e-02,1.175163e-06
bi-lstm_pred_0.006,6.426904e-199,1.160251e-28,4.967915e-12,6.307839e-152,2.260705e-13,6.436284e-13
bi-lstm_pred_0.007,6.141930e-188,1.696918e-47,6.391852e-15,8.848911e-20,4.882992e-09,5.614972e-13
lstm_pred_0.0,6.137345e-01,1.822750e-85,1.069866e-01,2.803731e-20,1.488752e-12,6.817060e-06
lstm_pred_0.001,3.452568e-137,1.079692e-81,2.283695e-08,5.386547e-228,2.062820e-29,1.361018e-02
lstm_pred_0.002,4.911547e-200,1.317084e-101,1.388613e-11,1.392860e-101,7.479463e-07,4.030379e-05


In [29]:
mcnemar_result.to_excel('./smtp-mcnemar-bi-lstm.xlsx')